## Pydantic

### why pydantic

In [ ]:
def insert_patient_data(name, age):

    print(f"Inserting patient data: Name={name}, Age={age}")

insert_patient_data("John Doe", 30)
    # This function would typically insert patient data into a database

print("-------------") # that's the problm
insert_patient_data("John Doe", "my_age")

Inserting patient data: Name=John Doe, Age=30
-------------
Inserting patient data: Name=John Doe, Age=my_age


In [ ]:
def insert_patient_data(name: str, age: int):

    print(f"Inserting patient data: Name={name}, Age={age}")

insert_patient_data("John Doe", 30)
    # This function would typically insert patient data into a database

print("-------------") # still's the problm
insert_patient_data("John Doe", "my_age")

Inserting patient data: Name=John Doe, Age=30
-------------
Inserting patient data: Name=John Doe, Age=my_age


In [ ]:
# this method is correct but as you know it is not scable in the productiv application 
def insert_patient_data(name: str, age: int):

    try:
        if type(name) == str and type(age) == int:
            print(f"Inserting patient data: Name={name}, Age={age}")
        else:
            raise TypeError("Invalid data types provided.")
    except TypeError as e:
        print(f"Error: {e}")

insert_patient_data("John Doe", "age")

## so that's why we need to use the pydantic in the data validation in the python 

Error: Invalid data types provided.


### How does pydantic work

**Pydantic**

1. Define a Pydantic **model** that represents the ideal **schema** of the data.
    * This includes the expected fields, their types, and any validation constraints (e.g., `gt=0` for positive numbers).
2. Instantiate the model with raw input data (usually a dictionary or JSON-like structure).
    * Pydantic will automatically **validate** the data and coerce it into the correct Python types (if possible).
    * If the data doesn't meet the model's requirements, Pydantic raises a `ValidationError`.
3. Pass the validated model object to functions or use it throughout your codebase.
    * This ensures that every part of your program works with **clean, type-safe, and logically valid data.**

In [12]:
from pydantic import BaseModel

class patient(BaseModel):
    name: str
    age: int
    

patient1 = patient(**{"name": "John Doe", "age": 30})
# print(patient1)

def insert_patient_data(patient: patient):

    print(f"Inserting patient data: Name={patient.name}, Age={patient.age}")

insert_patient_data(patient1)
    # This function would typically insert patient data into a database

# print("-------------") # that's the problm
# insert_patient_data("John Doe", "my_age")

Inserting patient data: Name=John Doe, Age=30


In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated

class patient(BaseModel):
    name: Annotated[str, Field(min_length=1, max_length=100, title="Patient Name", description="Full name of the patient")]
    age: int
    email: EmailStr
    linkedinurl: Optional[AnyUrl] = None
    weight: float = Field(gt=0, description="Weight in kilograms, must be non-negative", strict=True)
    married: Annotated[bool, Field(default=None, description="Is the patient married?")]
    allergies: List[str] = Field(max_length=22)
    contacts: Dict[str, str]
    alone: Optional[bool] = None
    

# ...existing code...
patient1 = patient(**{
    "name": "John Doe",
    "email": "badar@deeunlock.com",
    "linkedinurl": "https://www.linkedin.com/in/badarkhannn/",
    "age": 30,
    "weight": 23.2,
    "married": True, 
    "allergies": ["pollen", "nuts"], 
    "contacts": {"home": "123-456-7890", "work": "987-654-3210"}
})

# print(patient1)

def insert_patient_data(patient: patient):

    print(f"Inserting patient data: Name={patient.name}, Age={patient.age}")

insert_patient_data(patient1)
    # This function would typically insert patient data into a database

# print("-------------") # that's the problm
# insert_patient_data("John Doe", "my_age")

Inserting patient data: Name=John Doe, Age=30


### Field Validator

In [55]:
from pydantic import BaseModel, EmailStr, Field, field_validator
from typing import Dict, List

class patient(BaseModel):
    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allgergies: List[str]
    contact: Dict[str, str]

    @field_validator('email')
    @classmethod
    def email_validator(cls, value: EmailStr) -> EmailStr:
        valid_domains = ['deepunlock.com', 'microstun.com']
        domain = value.split('@')[1]
        if domain not in valid_domains:
            raise ValueError(f"Email domain must be one of {valid_domains}")
        return value
    
    @field_validator('name')
    @classmethod
    def tranformm_name(cls, value):
        return value.upper()

    @field_validator('age', mode='before')
    @classmethod
    def validate_age(cls, value):
        if 0 < value < 120:
            return value
        else:
            raise ValueError("Age must be between 0 and 120")


patient_info = {
    "name": "John Doe",
    "email": "badar@deepunlock.com",
    "age": 30,
    "weight": 23.2,
    "married": True,
    "allgergies": ["pollen", "nuts"],
    "contact": {"home": "123-456-7890", "work": "987-654-3210"}
}

patient1 = patient(**patient_info)

def insert_patient_data(patient: patient):
    print(f"Inserting patient data: Name={patient.name}, Age={patient.age}")

insert_patient_data(patient1) 

## ther is a two mode in teh field valider
# after and before
# after is used to validate the data after the data is created
# before is used to validate the data before the data is created

Inserting patient data: Name=JOHN DOE, Age=30


### Model validator

In [57]:
from pydantic import BaseModel, EmailStr, model_validator
from typing import List, Dict

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @model_validator(mode='after')
    def validate_emergency_contact(cls, model):
        if model.age > 60 and 'emergency' not in model.contact_details:
            raise ValueError('Patients older than 60 must have an emergency contact')
        return model



def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

patient_info = {'name':'john', 'email':'abc@example.com.com', 'age': '65', 'weight': 75.2, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}

patient1 = Patient(**patient_info) 

update_patient_data(patient1)

john
65
['pollen', 'dust']
True
updated


### commputed fields

In [58]:
from pydantic import BaseModel, EmailStr, computed_field
from typing import List, Dict

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float # kg
    height: float # mtr
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @computed_field
    @property
    def bmi(self) -> float:
        bmi = round(self.weight/(self.height**2),2)
        return bmi



def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('BMI', patient.bmi)
    print('updated')

patient_info = {'name':'nitish', 'email':'abc@icici.com', 'age': '65', 'weight': 75.2, 'height': 1.72, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}

patient1 = Patient(**patient_info) 

update_patient_data(patient1)

nitish
65
['pollen', 'dust']
True
BMI 25.42
updated


### NEsted Model

In [61]:
from pydantic import BaseModel

class Address(BaseModel):

    city: str
    state: str
    pin: str

class Patient(BaseModel):

    name: str
    gender: str
    age: int
    address: Address

address_dict = {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}

address1 = Address(**address_dict)

patient_dict = {'name': 'nitish', 'gender': 'male', 'age': 35, 'address': address1}

patient1 = Patient(**patient_dict)

temp = patient1.model_dump()

print(type(temp))


# Better organization of related data (e.g., vitals, address, insurance)

# Reusability: Use Vitals in multiple models (e.g., Patient, MedicalRecord)

# Readability: Easier for developers and API consumers to understand

# Validation: Nested models are validated automatically—no extra work needed

<class 'dict'>


### SErizalication

In [62]:
from pydantic import BaseModel

class Address(BaseModel):

    city: str
    state: str
    pin: str

class Patient(BaseModel):

    name: str
    gender: str = 'Male'
    age: int
    address: Address

address_dict = {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}

address1 = Address(**address_dict)

patient_dict = {'name': 'nitish', 'age': 35, 'address': address1}

patient1 = Patient(**patient_dict)

temp = patient1.model_dump(exclude_unset=True)

print(temp)
print(type(temp))

{'name': 'nitish', 'age': 35, 'address': {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}}
<class 'dict'>
